In [37]:
# Intendando coger la funcion de extraccion hasta este notebook, descargando nbconvert, y pasando el notebook1 en script python, pero sin conseguirlo. 
!jupyter nbconvert --to python Extraction.ipynb


[NbConvertApp] Converting notebook Extraction.ipynb to python
[NbConvertApp] Writing 5823 bytes to Extraction.py


In [7]:
from Extraction import download_excel
# Descarga del excel desde el notebook Extraction. 

download_excel()


- CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL.xlsx already exists.
- ESCUCHA_ACTIVA_DL.xlsx already exists.
- CST_GASTO_CONSUMO_TURISTICO_DL.xlsx already exists.
- ACTIVIDADES_OCIO_DL.xlsx already exists.

 CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'PAIS_ORIGEN', 'CIUDAD_DESTINO', 'TIPO_ORIGEN', 'BUSQUEDAS', 'ANTELACION', 'ESTANCIA_MEDIA', 'VARIACION_INTERANUAL_BUSQUEDAS']
First 3 rows:
    Unnamed: 0   AÑO  MES PAIS_ORIGEN CIUDAD_DESTINO    TIPO_ORIGEN  BUSQUEDAS  \
0           0  2022    1  Afganistán      Barcelona  Internacional         10   
1           1  2022    1  Afganistán         Madrid  Internacional         13   
2           2  2022    1  Afganistán         Málaga  Internacional          2   

  ANTELACION ESTANCIA_MEDIA VARIACION_INTERANUAL_BUSQUEDAS  
0       3,22           2,89                            0,0  
1       1,45           12,0                            0,0  
2        8,0            8,5                            0,0 

KeyError: 'FRONTUR_DL'

In [8]:
downloads = {
   # "FRONTUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/FRONTUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&Tipo%20de%20visitante=Todos&CCAA%20de%20destino=Todos",
   # "EGATUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/EGATUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&CCAA%20de%20destino=Todos",
   # "ACTIVIDADES_OCIO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ACTIVIDADES_OCIO_DL?CCAA=Todos&Provincia=Todos",
   # "TRANSPORTE_TERRESTRE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/TRANSPORTE_TERRESTRE_DL?desde%20%28a%C3%B1o%29=2018&Tipo%20transporte=Todos",
    "CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL?Pa%C3%ADs%20origen=Todos&Ciudad%20destino=Todos&Tipo%20origen=Todos",
    "ESCUCHA_ACTIVA_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ESCUCHA_ACTIVA_DL?Idioma=Todos&Medio=Todos&Sentimiento=Todos",
    "CST_GASTO_CONSUMO_TURISTICO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CST_GASTO_CONSUMO_TURISTICO_DL?Producto=Todos",
    "ACTIVIDADES_OCIO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ACTIVIDADES_OCIO_DL?CCAA=Todos&Provincia=Todos"

}

In [9]:
def download_excel(url, file_name, folder="./data"):
    file_path = os.path.join(folder, f"{file_name}.xlsx")

    if os.path.exists(file_path):
        print(f"- {file_name}.xlsx already exists.")
        return pd.read_excel(file_path)

    response = requests.get(url)
    
    if response.status_code == 200:
        file_type = response.headers["Content-Disposition"].split(".")[-1]

        if file_type == "xlsx":
            df = pd.read_excel(BytesIO(response.content))
            df.to_excel(file_path)
            print(f"{file_name}.xlsx downloaded and saved.")
            return df
        elif file_type == "csv":
            encoding = response.apparent_encoding
            df = pd.read_csv(StringIO(response.content.decode(encoding)), sep=';', engine='python')
            df.to_excel(file_path)
            print(f"{file_name}.csv downloaded and saved.")
            return df
    else:
        print(f"Failed to download {file_name}: {response.status_code}")
        return None

In [12]:
import os
import pandas as pd
datasets = {}

for name, url in downloads.items():
    df = download_excel(url, name)
    if df is not None:
        datasets[name] = df

- CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL.xlsx already exists.
- ESCUCHA_ACTIVA_DL.xlsx already exists.
- CST_GASTO_CONSUMO_TURISTICO_DL.xlsx already exists.
- ACTIVIDADES_OCIO_DL.xlsx already exists.


In [13]:
#Para visualizar datos en conjunto:

for name, df in datasets.items():
    print(f"\n {name}")
    print("Columns:", df.columns.tolist())
    print("First 3 rows:\n", df.head(3))


 CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'PAIS_ORIGEN', 'CIUDAD_DESTINO', 'TIPO_ORIGEN', 'BUSQUEDAS', 'ANTELACION', 'ESTANCIA_MEDIA', 'VARIACION_INTERANUAL_BUSQUEDAS']
First 3 rows:
    Unnamed: 0   AÑO  MES PAIS_ORIGEN CIUDAD_DESTINO    TIPO_ORIGEN  BUSQUEDAS  \
0           0  2022    1  Afganistán      Barcelona  Internacional         10   
1           1  2022    1  Afganistán         Madrid  Internacional         13   
2           2  2022    1  Afganistán         Málaga  Internacional          2   

  ANTELACION ESTANCIA_MEDIA VARIACION_INTERANUAL_BUSQUEDAS  
0       3,22           2,89                            0,0  
1       1,45           12,0                            0,0  
2        8,0            8,5                            0,0  

 ESCUCHA_ACTIVA_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'MENCIONES']
First 3 rows:
    Unnamed: 0   AÑO  MES  MENCIONES
0           0  2021    8      18136
1           1  2021    9      12554
2         

In [14]:
df_AOD = datasets["ACTIVIDADES_OCIO_DL"]
df_AOD

,AÑO,MES,PRODUCTO,CATEGORIA,CCAA,PROVINCIA,ENTRADAS,VISITAS_PAGINAS,GASTO_TOTAL,PRECIO_MEDIO_ENTRADA,TRANSACCIONES
0,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,35,135,420.0000,12.0000,12
1,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,2,8,6.4600,3.2300,1
2,2023,7,Espectaculos,Música,País Vasco,Araba/Álava,310,4171,3357.3000,10.8300,153
3,2023,7,Actividades,Actividades infantiles,País Vasco,Araba/Álava,39,151,207.8700,5.3300,17
4,2023,7,Espectaculos,"Cultura, teatro y danza",País Vasco,Araba/Álava,290,1123,2726.0000,9.4000,124
...,...,...,...,...,...,...,...,...,...,...,...
17207,2025,6,Actividades,Deportes y aventuras,Total Nacional,Total Nacional,3505,10282,99371.3065,28.3513,2250
17208,2025,6,Actividades,Actividades infantiles,Total Nacional,Total Nacional,10184,28670,189904.1032,18.6473,3980
17209,2025,6,Espectaculos,"Cultura, teatro y danza",Total Nacional,Total Nacional,38317,103158,550569.3096,14.3688,17599
17210,2025,6,Espectaculos,Museo y exposiciones,Total Nacional,Total Nacional,1086,3291,13221.1812,12.1742,324


In [22]:
#Esta tabla tiene muchas lineas, y vamos a limpiar esta tabla.  

def limpieza(df_AOD):
#La idea es : encontrar si hay lineas duplicadas y eliminarlas
    nb_antes = len(df_AOD)
    df_AOD = df_AOD.drop_duplicates()
    nb_despues = len(df_AOD)
    nb_borrado = nb_antes - nb_despues

    if nb_borrado > 0:
        print(f"{nb_borrado} lineas duplicadas borradas")
    else:
        print("No habia duplicado")

# Tambien vamos a eliminar el "total nacional" ya que nos bloquea para hacer las sumas.
    df_AOD = df_AOD[~df_AOD["CCAA"].str.upper().eq("TOTAL NACIONAL")]
        
#Las columnas de años, mes, producto, provincia, visita pagina, gasto toal, y transacciones no nos interesan para responder a nuestra pregunta.`
        
    columnas_a_borrar = ["AÑO", "MES", "PRODUCTO", "PROVINCIA", "VISITAS_PAGINAS", "GASTO_TOTAL", "TRANSACCIONES"]
    columnas =[col for col in columnas_a_borrar if col in df_AOD.columns]
    df_AOD = df_AOD.drop(columns=columnas)
    


#ESTRUCTURACION
#Añado una columna para tener los beneficios para cada categoria. 
      
    if "ENTRADAS" in df_AOD.columns and "PRECIO_MEDIO_ENTRADA" in df_AOD.columns:
        df_AOD["BENEFICIOS"] = df_AOD["ENTRADAS"] * df_AOD["PRECIO_MEDIO_ENTRADA"]
        print("Columna beneficios añadida")

    if "CATEGORIA" in df_AOD.columns and "BENEFICIOS" in df_AOD.columns:
        df_groupado = df_AOD.groupby(["CCAA","CATEGORIA"]).agg({
            "ENTRADAS": "sum",
            "PRECIO_MEDIO_ENTRADA": "mean",
            "BENEFICIOS": "sum"
        }).reset_index()
# Clasifico del mas grande beneficios al mas pequeño. 
    df_groupado = df_groupado.sort_values("BENEFICIOS", ascending=False)

    return df_groupado





In [23]:
limpieza(df_AOD)

13 lineas duplicadas borradas
Columna beneficios añadida


,CCAA,CATEGORIA,ENTRADAS,PRECIO_MEDIO_ENTRADA,BENEFICIOS
96,Comunidad de Madrid,Música,760693,17.236692,1.110072e+07
74,Cataluña,Música,696126,17.612375,1.010635e+07
137,Illes Balears,Música,488446,16.122196,8.914385e+06
10,Andalucía,Música,337814,17.185180,5.314741e+06
33,Canarias,Música,172083,19.527044,2.957131e+06
...,...,...,...,...,...
81,Comunidad Foral de Navarra,Cursos,8,42.466667,1.840000e+02
140,La Rioja,Cine,26,5.000000,1.300000e+02
167,Principado de Asturias,Museo y exposiciones,12,15.940000,9.068000e+01
161,Principado de Asturias,Cine,6,9.700000,5.820000e+01


En españa en entre 2023 y 2025, las actividades de ocio que hqn generado mas visitqs y mqs beneficios hqn sido : los eventos de Música en primero con mas de 3367086 entradas en el territorio español.  Seguido por la cultura, el teatro y la danza con mas de 1 075 900 entradas y en tercero por las actividades infantiles con mas de 474 683 entradas. 
